# PyTorch use prebuilt dataset

In [ ]:
# Import standard libraries that you may use most times
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from torchinfo import summary
from torchvision import transforms, datasets

print("PyTorch version:", torch.__version__)

In [ ]:
# Convert Pil image to PyTorch Tensor
data_transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
# Load the data set
mnist_train = datasets.MNIST(root='./data', train=True, transform=data_transform, download=True)
mnist_test  = datasets.MNIST(root='./data', train=False,transform=data_transform, download=True)

In [ ]:
# Prepare dataloaders
train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
test_dataloader  = torch.utils.data.DataLoader(mnist_test,  batch_size=64, shuffle=False)

# Explore the data

In [ ]:
# Get some random data
next(iter(train_dataloader))

In [ ]:
"Dataset size", len(mnist_train), len(mnist_test)

In [ ]:
"Shape of the samples", next(iter(train_dataloader))[0].shape, next(iter(train_dataloader))[1].shape

In [ ]:
# Plot
def plot_tensor(tensor, label, yh=""):
    plt.imshow(tensor.detach().numpy(), cmap='Greys')
    plt.title ("{}:^{}".format(label.item(),yh));
    plt.tick_params(left= False, labelleft = False , labelbottom = False, bottom = False)
    plt.show()
    
sample = next(iter(train_dataloader))
plot_tensor(sample[0][0].squeeze(), sample[1][0].squeeze())

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1     = nn.Linear(28 * 28, 128)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2     = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x) # Flatten tensor
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.softmax(self.fc2(x))
        return x
    
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model
model = NeuralNet().to(device)

# Print summary
summary(model, input_size=(1,28,28))

In [ ]:
# Predict labels using model (untrained)
predictions = model(sample[0][0].to(device)).cpu()
predictions

In [ ]:
# Plot prediction
p1 = torch.argmax(predictions)

plot_tensor(sample[0][0].squeeze(), sample[1][0].squeeze(),p1.item())

## Train and evaluate your model


In [ ]:
# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
CEloos   = nn.CrossEntropyLoss()

In [ ]:
# Fit the model
num_epochs = 5

# Iterate over #epochs
for epoch in range(num_epochs):
    # Keep track of network progress
    train_loss    = 0.0
    train_correct = 0
    test_correct  = 0
    
    # Visit each data sample once (random)
    for image, labels in train_dataloader: 
        # Compute model prediction and loss
        pred_labels = model(image.to(device))
        loss        = CEloos(pred_labels, labels.to(device))
        
        # Backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()       
        
        # Add loss to history
        train_loss    += loss.item()
        # Count number of correct predictions 
        train_correct += (torch.argmax(pred_labels.cpu(), 1) == labels.cpu()).sum().item()
    
    # Test loop (once per epoch)
    with torch.no_grad():
        for images, labels in test_dataloader:
            pred_labels = model(images.to(device))
            test_correct += (torch.argmax(pred_labels.cpu(), 1) == labels.cpu()).sum().item()
        
    # Compute accuracy (train & test)
    train_acc = train_correct / len(mnist_train)
    test_acc  = test_correct  / len(mnist_test)
    print('Epoch [{}/{}], Loss: {:.4f}, Train Acc: {:.2f}%, Test Acc: {:.2f}%'
          .format(epoch, num_epochs, train_loss / len(mnist_train), 100 * train_acc, 100 * test_acc))

In [ ]:
# Sample and plot
samples = next(iter(test_dataloader))[0][:5]
labels  = next(iter(test_dataloader))[1][:5]

In [ ]:
p = [torch.argmax(k) for k in model(samples.to(device))]
for i, l in enumerate(p):
    plt.figure(figsize=(1,1))
    plot_tensor(samples[i].squeeze(), labels[i].squeeze(), p[i].item())
    plt.show()


## Conclusion

Congratulations! on predicting minst data. You have successfully built a neural network to classify handwritten digits with Pytorch. You have achieved accuracy of 96%.